In [ ]:
import glob
import os
import pathlib
import parse_mom_sis_results as pmsr
import plot_pandas_data as ppd
import plot_right_axis_time as pra
from convert_time import days_to_gregorian_years, gregorian_years_to_days
from numpy import linspace

In [ ]:
log_basenames_025 = (
    "logfile.048x20.AVX2-REPRO.0.1m logfile.048x20.AVX2-REPRO.1.1m logfile.048x20.AVX2-REPRO.2.1m " + 
    "logfile.048x20.noht.AVX512.0.1m logfile.048x20.noht.AVX512.1.1m logfile.048x20.noht.AVX512.2.1m logfile.048x20.noht.AVX512.3.1m logfile.048x20.noht.AVX512.4.1m " +
    "logfile.048x20.noht.AVX512-REPRO.0.1m logfile.048x20.noht.AVX512-REPRO.1.1m logfile.048x20.noht.AVX512-REPRO.3.1m logfile.048x20.noht.AVX512-REPRO.4.1m logfile.048x20.noht.AVX512-REPRO.5.1m " +
    "logfile.048x40.AVX2-REPRO.0.1m logfile.048x40.AVX2-REPRO.1.1m logfile.048x40.AVX2-REPRO.2.1m " +
    "logfile.096x40.AVX2-REPRO.0.1m logfile.096x40.AVX2-REPRO.1.1m logfile.096x40.AVX2-REPRO.2.1m " + 
    "logfile.096x80.AVX2-REPRO.0.1m logfile.096x80.AVX2-REPRO.1.1m logfile.096x80.AVX2-REPRO.2.1m " +
    "logfile.192x80.AVX2-REPRO.2.1m logfile.192x80.AVX2-REPRO.3.1m logfile.192x80.AVX2-REPRO.4.1m " +
    "logfile.192x80.noht.AVX512.0.1m logfile.192x80.noht.AVX512.1.1m logfile.192x80.noht.AVX512.2.1m logfile.192x80.noht.AVX512.3.1m logfile.192x80.noht.AVX512.4.1m " +
    "logfile.192x80.noht.AVX512-REPRO.0.1m logfile.192x80.noht.AVX512-REPRO.1.1m logfile.192x80.noht.AVX512-REPRO.2.1m logfile.192x80.noht.AVX512-REPRO.3.1m"
)

In [ ]:
log_basenames_01 = (
    "logfile.0.1.048x20.AVX512-REPRO.0.1d logfile.0.1.048x40.AVX512-REPRO.0.1d logfile.0.1.096x40.AVX512-REPRO.0.1d logfile.0.1.096x80.AVX512-REPRO.0.1d logfile.0.1.192x80.AVX512-REPRO.0.1d " + 
    "logfile.0.1.048x20.AVX512-REPRO.1.1d logfile.0.1.048x40.AVX512-REPRO.1.1d logfile.0.1.096x40.AVX512-REPRO.1.1d logfile.0.1.096x80.AVX512-REPRO.1.1d logfile.0.1.192x80.AVX512-REPRO.1.1d " +
    "logfile.0.1.048x20.AVX512-REPRO.2.1d logfile.0.1.048x40.AVX512-REPRO.2.1d logfile.0.1.096x40.AVX512-REPRO.2.1d logfile.0.1.096x80.AVX512-REPRO.2.1d logfile.0.1.192x80.AVX512-REPRO.2.1d"
)

In [ ]:
mom_sis_work_dir = pathlib.PurePath("../access-om2/mom-sis/src/mom/work/")
log_dirname_025_bw_cl = os.path.join(mom_sis_work_dir,"global_0.25_degree_NYF.BW-vs-CL","logs.1m")
log_dirname_025 = os.path.join(mom_sis_work_dir,"global_0.25_degree_NYF","logs.1m")
log_dirname_01 = os.path.join(mom_sis_work_dir,"global_0.1_degree","logs.1d")

In [ ]:
bw_cl_globname = "logfile.0?8x?8.*.1m"
logfiles_025_bw_cl = glob.glob(os.path.join(log_dirname_025_bw_cl, bw_cl_globname))
logfiles_025 = [os.path.join(log_dirname_025, basename) for basename in log_basenames_025.split()]
logfiles_01 = [os.path.join(log_dirname_01, basename) for basename in log_basenames_01.split()]

In [ ]:
df_025_bw_cl = pmsr.parse_all(logfiles_025_bw_cl)
df_025 = pmsr.parse_all(logfiles_025)
df_01 = pmsr.parse_all(logfiles_01)

In [ ]:
ppd.semilogx_by_group("cols", "Ocean speed", 
    by="arch", 
    data=df_025_bw_cl,
    xlabel="MOM-SIS NPROCY",
    ylabel="Ocean speed (d/d)",
    yticks = linspace(0,10000,6),
    right_yticks = linspace(0,25,6),                  
    right_axis_fn=pra.plot_right_axis_yd,
    legend_title="Architecture")

In [ ]:
mean_speed=dict()
group_arch=df_025_bw_cl.groupby("arch")
for name, group in group_arch:
    means = group.groupby("cols")["Ocean speed"].mean()
    mean_speed[name] = means
for cols in [28,48]:
    for arch in ["AVX2-REPRO-CL","AVX512-REPRO","AVX512"]:
        speed_cl_vs_bw = mean_speed[arch][cols] / mean_speed["AVX2-REPRO-BW"][cols]
        print(f"{arch:14} {cols} {speed_cl_vs_bw:-6.4}")

In [ ]:
ppd.loglog_by_group("ncpus", "Ocean speed", 
    by="arch", 
    data=df_025,
    xlabel="MOM-SIS cores",
    ylabel="Ocean speed (d/d)",
    yticks = [5000,10000,20000,50000,100000],
    right_yticks=[10,20,50,100,200],                
    right_axis_fn=pra.plot_right_axis_yd,
    legend_title="Architecture")

In [ ]:
ppd.semilogx_by_group("ncpus", "Ocean speed per cpu", 
    by="arch", 
    data=df_025,
    xlabel="MOM-SIS cores",
    ylabel="Ocean speed (d/d) per core",
    yticks=range(8),
    right_yticks=[n/200.0 for n in range(4)],
    right_axis_fn=pra.plot_right_axis_yd,
    legend_title="Architecture")

In [ ]:
mean_025_speed=dict()
group_025_arch=df_025.groupby("arch")
for name, group in group_025_arch:
    means = group.groupby("ncpus")["Ocean speed"].mean()
    mean_025_speed[name] = means
speed_lo = mean_025_speed["AVX512-REPRO"][960] / mean_025_speed["AVX2-REPRO"][960]
speed_hi = mean_025_speed["AVX512-REPRO"][15360] / mean_025_speed["AVX2-REPRO"][15360]
print(round(speed_lo,2), round(days_to_gregorian_years(speed_lo),4))
print(round(speed_hi,2), round(days_to_gregorian_years(speed_hi),4))

In [ ]:
xticks_025 = df_025["ncpus"].sort_values().unique() 
xticks_01 = df_01["ncpus"].sort_values().unique()

In [ ]:
ppd.loglog("ncpus","Ocean speed",
    data=df_01,
    xlabel="MOM-SIS cores",
    ylabel="Ocean speed (d/d)",
    xticks=xticks_01,
    yticks=[100,200,500,1000,2000],
    right_yticks=[0.5,1,2,5,10],
    right_axis_fn=pra.plot_right_axis_yd)

In [ ]:
ppd.semilogx("ncpus","Ocean speed per cpu",
    data=df_01,
    xlabel="MOM-SIS cores",
    ylabel="Ocean speed (d/d) per core",
    xticks=xticks_01,
    yticks=[n/20.0 for n in range(5)],
    right_yticks=[n/10000.0 for n in range(6)],
    right_axis_fn=pra.plot_right_axis_yd)

In [ ]:
ppd.loglog_by_group("ncpus","ocean_time_per_step",
    by="arch",
    data=df_025,
    xlabel="MOM-SIS cores",
    ylabel="Ocean time per step (s)",
    yticks=[0.02,0.05,0.1,0.2,0.5],
    right_yticks=[],
    right_axis_fn=pra.no_right_axis,
    legend_title="Architecture",
    marker='d')

In [ ]:
ppd.semilogx_by_group("ncpus","ocean_cpu_time_per_step",
    by="arch",
    data=df_025,
    xlabel="MOM-SIS cores",
    ylabel="MOM-SIS core time per step (core s)",
    yticks=[100.0 * n for n in range(7)],
    right_yticks=[n/50.0 for n in range(9)],
    right_axis_fn=pra.plot_right_axis_sh,
    legend_title="Architecture",
    marker='d')

In [ ]:
ppd.loglog("ncpus","ocean_time_per_step",
    data=df_01,
    xlabel="MOM-SIS cores",
    ylabel="Ocean time per step (s)",
    xticks=xticks_01,
    yticks=[0.1,0.2,0.5,1.0,2.0],
    right_yticks=[],
    right_axis_fn=pra.no_right_axis,
    marker='d')

In [ ]:
ppd.semilogx("ncpus","ocean_cpu_time_per_step",
    data=df_01,
    xlabel="MOM-SIS cores",
    ylabel="MOM-SIS core time per step (core s)",
    xticks=xticks_01,
    yticks=[500.0 * n for n in range(6)],
    right_yticks=[n/10.0 for n in range(8)],
    right_axis_fn=pra.plot_right_axis_sh,
    marker='d')             

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
df_025_bw_cl

In [ ]:
df_025

In [ ]:
df_01

In [ ]:
import matplotlib
matplotlib.__version__